In [1]:
# Instalar las dependencias necesarias
!apt-get install -y ffmpeg
!pip install tensorflow
!pip install soundfile
!pip install scikit-learn
!pip install transformers
!pip install xgboost
!pip install resampy

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
!pip install tf_slim

In [3]:
# Crear el directorio vggish
!mkdir -p vggish

# Descargar los archivos necesarios
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/vggish/vggish_input.py -P vggish/
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/vggish/vggish_params.py -P vggish/
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/vggish/vggish_postprocess.py -P vggish/
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/vggish/vggish_slim.py -P vggish/
!wget https://storage.googleapis.com/audioset/vggish_model.ckpt -P vggish/
!wget https://storage.googleapis.com/audioset/vggish_pca_params.npz -P vggish/

--2024-07-17 02:25:27--  https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/vggish/vggish_input.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3536 (3.5K) [text/plain]
Saving to: ‘vggish/vggish_input.py.5’

vggish_input.py.5   100%[===================>]   3.45K  --.-KB/s    in 0s      

2024-07-17 02:25:28 (31.3 MB/s) - ‘vggish/vggish_input.py.5’ saved [3536/3536]

--2024-07-17 02:25:28--  https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/vggish/vggish_params.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, 

In [4]:
import os
import numpy as np
import soundfile as sf
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import pickle


import sys
sys.path.append('./vggish')
import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim

2024-07-17 02:25:38.556924: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 02:25:38.653433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 02:25:38.653971: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 02:25:38.805453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 02:25:40.325041: W tensorflow/com

In [5]:
# Paths to the dataset directories
train_dir = "/teamspace/studios/this_studio/dataset/train"
test_dir = "/teamspace/studios/this_studio/dataset/test"
checkpoint_dir = "/teamspace/studios/this_studio/checkpointvg"
os.makedirs(checkpoint_dir, exist_ok=True)

In [30]:
test2_dir = "/teamspace/studios/this_studio/chord_examples/base de datos/test"
train2_dir = "/teamspace/studios/this_studio/chord_examples/base de datos/train"
test3_dir = "/teamspace/studios/this_studio/chord_examples/base de datos 3/test"
train3_dir = "/teamspace/studios/this_studio/chord_examples/base de datos 3/train"

In [27]:
 #Define some constants
BATCH_SIZE = 128

def process_audio_file(file_path):
    examples_batch = vggish_input.wavfile_to_examples(file_path)
    features = np.mean(examples_batch, axis=0)  # Average the features over all segments
    return features.flatten()  # Ensure the output is a 1D vector

In [28]:
def load_data_from_directory(directory):
    features, labels = [], []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            label = os.path.basename(root)
            audio_features = process_audio_file(file_path)
            features.append(audio_features)
            labels.append(label)
    return features, labels

In [29]:
def process_batches(directory, checkpoint_file=None):
    if checkpoint_file and os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'rb') as f:
            return pickle.load(f)
    
    features, labels = [], []
    for root, _, files in os.walk(directory):
        for i, file in enumerate(files):
            file_path = os.path.join(root, file)
            label = os.path.basename(root)
            audio_features = process_audio_file(file_path)
            features.append(audio_features)
            labels.append(label)
            if (i + 1) % BATCH_SIZE == 0:
                with open(checkpoint_file, 'wb') as f:
                    pickle.dump((features, labels), f)
    
    with open(checkpoint_file, 'wb') as f:
        pickle.dump((features, labels), f)
    
    return features, labels

In [9]:
# Paths to your checkpoints
train_checkpoint = os.path.join(checkpoint_dir, "train_features.pkl")
test_checkpoint = os.path.join(checkpoint_dir, "test_features.pkl")

# Process train and test data
train_features, train_labels = process_batches(train_dir, checkpoint_file=train_checkpoint)
test_features, test_labels = process_batches(test_dir, checkpoint_file=test_checkpoint)

# Convert lists to numpy arrays
train_features = np.array(train_features)
test_features = np.array(test_features)

# # Transform the labels into a numerical format
label_to_int = {label: idx for idx, label in enumerate(set(train_labels))}
train_labels = [label_to_int[label] for label in train_labels]
test_labels = [label_to_int[label] for label in test_labels]

# Save the label mapping
label_mapping_path = os.path.join(checkpoint_dir, "label_mapping.pkl")
with open(label_mapping_path, 'wb') as f:
    pickle.dump(label_to_int, f)


In [51]:
# Datos de prueba base de datos
train2_checkpoint = os.path.join(checkpoint_dir, "train2_features.pkl")
test2_checkpoint = os.path.join(checkpoint_dir, "test2_features.pkl")

# Process train and test data
train_features2, train_labels2 = process_batches(train2_dir, checkpoint_file=train2_checkpoint)
test_features2, test_labels2 = process_batches(test2_dir, checkpoint_file=test2_checkpoint)

# Convert lists to numpy arrays
train_features2 = np.array(train_features2)
test_features2 = np.array(test_features2)

In [64]:
# Datos de prueba base de datos 3
train3_checkpoint = os.path.join(checkpoint_dir, "train3_features.pkl")
test3_checkpoint = os.path.join(checkpoint_dir, "test3_features.pkl")

# Process train and test data
train_features3, train_labels3 = process_batches(train3_dir, checkpoint_file=train3_checkpoint)
test_features3, test_labels3 = process_batches(test3_dir, checkpoint_file=test3_checkpoint)

# Convert lists to numpy arrays
train_features3 = np.array(train_features3)
test_features3 = np.array(test_features3)

In [6]:
train_checkpoint = os.path.join(checkpoint_dir, "train_features.pkl")

# Load the training features and labels
with open(train_checkpoint, 'rb') as f:
    train_features, train_labels = pickle.load(f)

In [7]:
test_checkpoint = os.path.join(checkpoint_dir, "test_features.pkl")

# Load the training features and labels
with open(test_checkpoint, 'rb') as f:
    test_features, test_labels = pickle.load(f)

In [8]:
# Convert lists to numpy arrays
train_features = np.array(train_features)
train_labels = np.array(train_labels)

In [9]:
# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

In [10]:


# Define parameter grids for each model
svm_param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
rf_param_grid = {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]}
xgb_param_grid = {'n_estimators': [100, 200], 'max_depth': [3, 6, 10], 'learning_rate': [0.01, 0.1, 0.2]}

# Define the models
svm_model = SVC()
rf_model = RandomForestClassifier()
xgb_model = XGBClassifier()

# Perform grid search
svm_grid = GridSearchCV(svm_model, svm_param_grid, cv=5, n_jobs=-1, verbose=2)
rf_grid = GridSearchCV(rf_model, rf_param_grid, cv=5, n_jobs=-1, verbose=2)
xgb_grid = GridSearchCV(xgb_model, xgb_param_grid, cv=5, n_jobs=-1, verbose=2)

In [11]:
# Train the models
print("Training SVM...")
svm_grid.fit(X_train, y_train)
print("Best SVM Parameters:", svm_grid.best_params_)

Training SVM...
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[CV] END ...............................C=0.1, kernel=linear; total time=13.2min
[CV] END ...............................C=0.1, kernel=linear; total time=13.2min
[CV] END ...............................C=0.1, kernel=linear; total time=13.2min
[CV] END ...............................C=0.1, kernel=linear; total time=13.3min
[CV] END ...............................C=0.1, kernel=linear; total time=11.7min
[CV] END ..................................C=0.1, kernel=rbf; total time=98.4min
[CV] END ..................................C=0.1, kernel=rbf; total time=99.6min
[CV] END .................................C=0.1, kernel=rbf; total time=100.3min
[CV] END .................................C=1, kernel=linear; total time=10.5min
[CV] END .................................C=1, kernel=linear; total time=10.8min
[CV] END ..................................C=0.1, kernel=rbf; total time=99.5min
[CV] END .................................C=1, kernel=linear; total time=11.6min
[CV] END ...................

In [10]:
svm_model = SVC(kernel='linear',C=0.1)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_val)
# Calcular la precisión del modelo
accuracy = accuracy_score(y_val, y_pred)
print(f'Precisión del modelo: {accuracy * 100:.2f}%')

Precisión del modelo: 100.00%


In [12]:
print("Training Random Forest...")
rf_grid.fit(X_train, y_train)
print("Best Random Forest Parameters:", rf_grid.best_params_)

Training Random Forest...
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[CV] END ...................max_depth=None, n_estimators=100; total time= 8.4min
[CV] END ...................max_depth=None, n_estimators=100; total time= 8.5min
[CV] END ...................max_depth=None, n_estimators=100; total time= 8.5min
[CV] END ...................max_depth=None, n_estimators=100; total time= 8.6min
[CV] END ...................max_depth=None, n_estimators=100; total time= 8.3min
[CV] END ...................max_depth=None, n_estimators=200; total time=16.7min
[CV] END ...................max_depth=None, n_estimators=200; total time=17.0min
[CV] END ...................max_depth=None, n_estimators=200; total time=17.0min
[CV] END .....................max_depth=10, n_estimators=100; total time= 5.7min
[CV] END .....................max_depth=10, n_estimators=100; total time= 5.7min
[CV] END ...................max_depth=None, n_estimators=200; total time=16.7min
[CV] END .....................max_depth=10, n_estimators=100; total time= 5.7min
[CV] END ...................

In [11]:
rf_model = RandomForestClassifier(max_depth=None,n_estimators=100)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)
# Calcular la precisión del modelo
accuracy = accuracy_score(y_val, y_pred)
print(f'Precisión del modelo: {accuracy * 100:.2f}%')

Precisión del modelo: 100.00%


In [15]:
print("Training XGBoost...")
xgb_grid.fit(X_train, y_train)
print("Best XGBoost Parameters:", xgb_grid.best_params_)

Training XGBoost...
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.9s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.9s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.9s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.9s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.9s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   0.9s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   1.0s
[CV] END ..learning_rate=0.0

ValueError: 
All the 90 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/xgboost/core.py", line 729, in inner_f
    return func(**kwargs)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/xgboost/sklearn.py", line 1467, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 3

In [ ]:
# Evaluate the models on the validation set
print("SVM Validation Accuracy:", accuracy_score(y_val, svm_grid.predict(X_val)))
print("Random Forest Validation Accuracy:", accuracy_score(y_val, rf_grid.predict(X_val)))
print("XGBoost Validation Accuracy:", accuracy_score(y_val, xgb_grid.predict(X_val)))

In [13]:
# Save the models
svm_checkpoint = os.path.join(checkpoint_dir, "svm_model.pkl")
rf_checkpoint = os.path.join(checkpoint_dir, "rf_model.pkl")
#xgb_checkpoint = os.path.join(checkpoint_dir, "xgb_model.pkl")

with open(svm_checkpoint, 'wb') as f:
    pickle.dump(svm_grid.best_estimator_, f)
with open(rf_checkpoint, 'wb') as f:
    pickle.dump(rf_grid.best_estimator_, f)

In [24]:

# with open(xgb_checkpoint, 'wb') as f:
#     pickle.dump(xgb_grid.best_estimator_, f)

#Evaluate the models on the test set
svm_pred = svm_grid.predict(test_features)
rf_pred = rf_grid.predict(test_features)
# xgb_pred = xgb_grid.predict(test_features)

print("SVM Test Classification Report")
print(classification_report(test_labels, svm_pred))

print("Random Forest Test Classification Report")
print(classification_report(test_labels, rf_pred))

# print("XGBoost Test Classification Report")
# print(classification_report(test_labels, xgb_pred))

SVM Test Classification Report
                    precision    recall  f1-score   support

       DoAumentada       1.00      1.00      1.00       270
      DoDisminuida       1.00      1.00      1.00       270
           DoMayor       1.00      1.00      1.00       270
           DoMenor       1.00      1.00      1.00       270
       FaAumentada       1.00      1.00      1.00       270
      FaDisminuida       1.00      1.00      1.00       270
           FaMayor       1.00      1.00      1.00       270
           FaMenor       1.00      1.00      1.00       270
       LaAumentada       1.00      1.00      1.00       270
  LaBemolAumentada       1.00      1.00      1.00       270
 LaBemolDisminuida       1.00      1.00      1.00       270
      LaBemolMayor       1.00      1.00      1.00       270
      LaBemolMenor       1.00      1.00      1.00       270
      LaDisminuida       1.00      1.00      1.00       270
           LaMayor       1.00      1.00      1.00       270
        

In [26]:
print("Random Forest Test Classification Report")
print(classification_report(test_labels, rf_pred))

Random Forest Test Classification Report
                    precision    recall  f1-score   support

       DoAumentada       1.00      1.00      1.00       270
      DoDisminuida       1.00      1.00      1.00       270
           DoMayor       1.00      1.00      1.00       270
           DoMenor       1.00      1.00      1.00       270
       FaAumentada       1.00      1.00      1.00       270
      FaDisminuida       1.00      1.00      1.00       270
           FaMayor       1.00      1.00      1.00       270
           FaMenor       1.00      1.00      1.00       270
       LaAumentada       1.00      1.00      1.00       270
  LaBemolAumentada       1.00      1.00      1.00       270
 LaBemolDisminuida       1.00      1.00      1.00       270
      LaBemolMayor       1.00      1.00      1.00       270
      LaBemolMenor       1.00      1.00      1.00       270
      LaDisminuida       1.00      1.00      1.00       270
           LaMayor       1.00      1.00      1.00       27

In [45]:
#Evaluate the models on the test set
svm_pred = svm_grid.predict(train_features2)
rf_pred = rf_grid.predict(train_features2)

In [53]:
print("SVM Test Classification Report")
print(classification_report(train_labels2, svm_pred))

SVM Test Classification Report
                    precision    recall  f1-score   support

       DoAumentada       0.00      0.00      0.00         0
      DoDisminuida       0.00      0.00      0.00         0
           DoMayor       1.00      0.06      0.12        16
           DoMenor       0.14      0.06      0.09        16
       FaAumentada       0.00      0.00      0.00         0
      FaDisminuida       0.00      0.00      0.00         0
           FaMayor       0.00      0.00      0.00        16
           FaMenor       0.00      0.00      0.00        16
 LaBemolDisminuida       0.00      0.00      0.00         0
      LaBemolMayor       0.71      0.62      0.67        16
      LaBemolMenor       0.00      0.00      0.00        16
           LaMayor       0.00      0.00      0.00        16
           LaMenor       0.00      0.00      0.00        16
       MiAumentada       0.00      0.00      0.00         0
  MiBemolAumentada       0.00      0.00      0.00         0
 MiBemol

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [55]:
print("Random Forest Test Classification Report")
print(classification_report(train_labels2, rf_pred))

Random Forest Test Classification Report
                    precision    recall  f1-score   support

      DoDisminuida       0.00      0.00      0.00         0
           DoMayor       0.00      0.00      0.00        16
           DoMenor       0.00      0.00      0.00        16
       FaAumentada       0.00      0.00      0.00         0
           FaMayor       0.00      0.00      0.00        16
           FaMenor       0.00      0.00      0.00        16
 LaBemolDisminuida       0.00      0.00      0.00         0
      LaBemolMayor       0.00      0.00      0.00        16
      LaBemolMenor       0.00      0.00      0.00        16
           LaMayor       0.00      0.00      0.00        16
           LaMenor       0.00      0.00      0.00        16
       MiAumentada       0.00      0.00      0.00         0
      MiBemolMayor       0.00      0.00      0.00        16
      MiBemolMenor       0.00      0.00      0.00        16
           MiMayor       0.00      0.00      0.00        1

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [56]:
#Evaluate the models on the test set
svm_pred = svm_grid.predict(test_features2)
rf_pred = rf_grid.predict(test_features2)

In [58]:
print("SVM Test Classification Report")
print(classification_report(test_labels2, svm_pred))

SVM Test Classification Report
                   precision    recall  f1-score   support

     DoDisminuida       0.00      0.00      0.00         0
          DoMayor       0.00      0.00      0.00         4
          DoMenor       0.00      0.00      0.00         4
      FaAumentada       0.00      0.00      0.00         0
          FaMayor       0.00      0.00      0.00         4
          FaMenor       0.00      0.00      0.00         4
     LaBemolMayor       0.75      0.75      0.75         4
     LaBemolMenor       0.00      0.00      0.00         4
          LaMayor       0.00      0.00      0.00         4
          LaMenor       0.00      0.00      0.00         4
      MiAumentada       0.00      0.00      0.00         0
 MiBemolAumentada       0.00      0.00      0.00         0
MiBemolDisminuida       0.00      0.00      0.00         0
     MiBemolMayor       0.00      0.00      0.00         4
     MiBemolMenor       0.00      0.00      0.00         4
     MiDisminuida       

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [60]:
print("Random Forest Test Classification Report")
print(classification_report(test_labels2, rf_pred))

Random Forest Test Classification Report
                    precision    recall  f1-score   support

      DoDisminuida       0.00      0.00      0.00         0
           DoMayor       0.00      0.00      0.00         4
           DoMenor       0.00      0.00      0.00         4
       FaAumentada       0.00      0.00      0.00         0
           FaMayor       0.00      0.00      0.00         4
           FaMenor       0.00      0.00      0.00         4
      LaBemolMayor       0.00      0.00      0.00         4
      LaBemolMenor       0.00      0.00      0.00         4
           LaMayor       0.00      0.00      0.00         4
           LaMenor       0.00      0.00      0.00         4
      MiBemolMayor       0.00      0.00      0.00         4
      MiBemolMenor       0.00      0.00      0.00         4
           MiMayor       0.00      0.00      0.00         4
           MiMenor       0.00      0.00      0.00         4
 ReBemolDisminuida       0.00      0.00      0.00         

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [65]:
#Evaluate the models on the test set
svm_pred = svm_grid.predict(train_features3)
rf_pred = rf_grid.predict(train_features3)

In [66]:
print("SVM Test Classification Report")
print(classification_report(train_labels3, svm_pred))

SVM Test Classification Report
                    precision    recall  f1-score   support

       DoAumentada       0.00      0.00      0.00         0
      DoDisminuida       0.00      0.00      0.00         0
           DoMayor       0.50      0.50      0.50         8
           DoMenor       0.23      0.38      0.29         8
      FaDisminuida       0.00      0.00      0.00         0
           FaMayor       1.00      0.25      0.40         8
           FaMenor       0.86      0.75      0.80         8
       LaAumentada       0.00      0.00      0.00         0
      LaBemolMayor       0.80      0.50      0.62         8
      LaBemolMenor       1.00      0.62      0.77         8
           LaMayor       0.71      0.62      0.67         8
           LaMenor       1.00      0.38      0.55         8
       MiAumentada       0.00      0.00      0.00         0
  MiBemolAumentada       0.00      0.00      0.00         0
      MiBemolMayor       0.67      0.25      0.36         8
      Mi

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

In [67]:
print("Random Forest Test Classification Report")
print(classification_report(train_labels3, rf_pred))

Random Forest Test Classification Report
                   precision    recall  f1-score   support

      DoAumentada       0.00      0.00      0.00         0
     DoDisminuida       0.00      0.00      0.00         0
          DoMayor       0.00      0.00      0.00         8
          DoMenor       0.00      0.00      0.00         8
      FaAumentada       0.00      0.00      0.00         0
     FaDisminuida       0.00      0.00      0.00         0
          FaMayor       1.00      0.25      0.40         8
          FaMenor       0.18      0.75      0.29         8
LaBemolDisminuida       0.00      0.00      0.00         0
     LaBemolMayor       0.50      0.62      0.56         8
     LaBemolMenor       0.83      0.62      0.71         8
          LaMayor       0.42      0.62      0.50         8
          LaMenor       1.00      0.62      0.77         8
 MiBemolAumentada       0.00      0.00      0.00         0
     MiBemolMayor       1.00      0.12      0.22         8
     MiBemolMe

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(